In [1]:
import pandas as pd
import json
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import openai
import random
from retrying import retry
import pyarrow as pa
import pyarrow.parquet as pq
from transformers import AutoTokenizer
import torch
from modelscope import GenerationConfig
from modelscope import AutoModelForCausalLM, AutoTokenizer, snapshot_download

openai.api_base = "https://one.aiskt.com/v1"
model = "gpt-3.5-turbo"


class OpenAIGPT:
    def __init__(self, model_name="gpt-3.5-turbo", keys_path=None):
        self.model_name = model_name
        with open(keys_path, encoding="utf-8", mode="r") as fr:
            self.keys = [line.strip() for line in fr if len(line.strip()) >= 4]

    def __post_process(self, response):
        return response["choices"][0]["message"]["content"]

    @retry(wait_fixed=300, stop_max_attempt_number=50)
    def __call__(self, message):
        if message is None or message == "":
            return False, "Your input is empty."

        # current_key = random.choice(self.keys)
        current_key = self.keys[0] if len(self.keys) == 1 else random.choice(self.keys)
        openai.api_key = current_key
        response = openai.ChatCompletion.create(
            model=self.model_name,
            messages=[{"role": "user", "content": message}],
            temperature=0.3,
            top_p=0.7,
            frequency_penalty=0.6,
            presence_penalty=0.6,
            n=1,
            max_tokens=10
        )
        return self.__post_process(response)


igpt = OpenAIGPT(keys_path="../train/apikeys.txt")

2024-05-16 19:40:36,656 - modelscope - INFO - PyTorch version 2.2.1 Found.
2024-05-16 19:40:36,765 - modelscope - INFO - Loading ast index from /home/zhangmin/.cache/modelscope/ast_indexer
2024-05-16 19:40:36,765 - modelscope - INFO - Loading ast index from /home/zhangmin/.cache/modelscope/ast_indexer
2024-05-16 19:40:36,811 - modelscope - INFO - Loading done! Current index file version is 1.13.3, with md5 f225825bf0706e4b6d2ef5df7f1df86c and a total number of 972 components indexed


In [2]:
igpt('hello')

'Hello! How can I assist you today?'

In [3]:
data = pd.read_json('ft_data_summary_new2_test.json')

In [4]:
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import pandas as pd


def process_row(item):
    def predict(prompt):
        global igpt
        return igpt(prompt)

    text_input = f'''{item['prompt']}你只需要输出一个选项即可，不需要做任何分许判断\n{item['content']}'''
    predicted_response = predict(text_input).replace(" ", "")

    expected_response = item['label']
    result = {
        "prompt": item['prompt'],
        "content": item['content'],
        "expected": expected_response,
        "predicted": predicted_response
    }
    return result


def generate_dataset(df, max_samples=32, output_file_path='test_dataset/gpt.json'):
    df_subset = df.head(max_samples)
    results = []

    with ThreadPoolExecutor(max_workers=50) as executor:
        futures = [executor.submit(process_row, row) for _, row in df_subset.iterrows()]

        for future in tqdm(as_completed(futures), total=len(futures), desc="Processing rows"):
            result = future.result()
            if result:
                results.append(result)
                with open(output_file_path, 'w', encoding='utf-8') as f:
                    json.dump(results, f, ensure_ascii=False, indent=2)

    return results


processed_data = generate_dataset(data, max_samples=1400)

Processing rows: 100%|██████████| 1395/1395 [02:27<00:00,  9.45it/s]


In [5]:
# import pandas as pd
# import torch
# from modelscope import GenerationConfig
# from modelscope import AutoModelForCausalLM, AutoTokenizer, snapshot_download
# from tqdm import tqdm
# import json

# # from llmtuner import ChatModel
# # model_path = "/home/zhangmin/toby/Quant-GPT/exp_model/v3_1"
# # # model_path = "/home/zhangmin/.cache/modelscope/hub/TongyiFinance/Tongyi-Finance-14B-Chat"
# # model = ChatModel(dict(
# #     model_name_or_path=model_path,
# #     template="qwen",
# #     temperature=0.3,
# #     top_p=0.6,
# #     cutoff_len=8192,
# #     max_length=256,
# #     do_sample=True
# #     # infer_backend="vllm",
# #     # vllm_gpu_util=0.7
# # ))


# def predict(prompt):
#     return igpt(prompt)


# def evaluate_accuracy(test_data):
#     results = []
#     correct_predictions = 0
#     total_predictions = 0
#     output_file = 'test_dataset/gpt.json'

#     with tqdm(total=test_data.shape[0]) as pbar:
#         for _, item in test_data.iterrows():
#             text_input = f'''{item['prompt']}你只需要输出一个选项即可，不需要做任何分许判断\n{item['content']}'''
#             predicted_response = predict(text_input).replace(" ", "")

#             expected_response = item['label']
#             result = {
#                 "prompt": item['prompt'],
#                 "content": item['content'],
#                 "expected": expected_response,
#                 "predicted": predicted_response
#             }
#             results.append(result)
#             total_predictions += 1
#             if expected_response == predicted_response:
#                 correct_predictions += 1

#             accuracy = correct_predictions / total_predictions
#             pbar.set_description(f"{correct_predictions}/{total_predictions}, {predicted_response}, {expected_response}, {expected_response == predicted_response}")
#             pbar.update()

#             # 实时更新JSON文件
#             with open(output_file, 'w') as f:
#                 json.dump(results, f, indent=4, ensure_ascii=False)

#     return accuracy


# max_samples = 10
# test_data = pd.read_json('ft_data_summary_new_test.json')
# # test_data = test_data.sample(frac=1).reset_index(drop=True)
# test_data = test_data[:max_samples]

# accuracy = evaluate_accuracy(test_data)
# print("Accuracy:", accuracy)